In [2]:
# ---- 1. Importação das Bibliotecas ----
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
import warnings

warnings.filterwarnings('ignore') #para ignorar avisos comuns de bibliotecas
print('Bibliotecas importadas com sucesso!')

Bibliotecas importadas com sucesso!


In [7]:
# ---- 2. Carregando o Dataset e Fazendo Exploração Inicial dos Dados ----
url = "https://raw.githubusercontent.com/selva86/datasets/master/bank-additional-full.csv"
try:
    data = pd.read_csv(url, sep=';')
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os dados: {e}")

Dados carregados com sucesso!


In [8]:
# ---- 3. Explorando os Dados ----
print("--- Amostra dos Dados ---")
data.head()

--- Amostra dos Dados ---


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [9]:
# ---- 4. Explorando as iformaçõies dos Dados ----
print("--- Informações das Colunas ---")
data.info()

--- Informações das Colunas ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17 

In [10]:
# ---- 5. Explorando o Alvo (y) ----
# Vamos verificar a proporção de 'yes' (converteu) e 'no' (não converteu).
print("--- Distribuição do Alvo (Conversão 'y'): ---")
print(data['y'].value_counts(normalize=True))

--- Distribuição do Alvo (Conversão 'y'): ---
y
no     0.887346
yes    0.112654
Name: proportion, dtype: float64


In [14]:
# ---- 6. Definição das Features (x) e do Alvo (y) ----
X = data.drop('y', axis=1)
y = data['y']

#Converter o alvo 'yes/no' para 1/0.
y = y.map({'yes':1, 'no': 0})

print(f"Formato das Features (X): {X.shape}")
print(f"Formato do Alvo (y): {y.shape}")

Formato das Features (X): (41188, 20)
Formato do Alvo (y): (41188,)


In [20]:
# ---- 7. Pré-processamento (Indetificação das Colunas) ----
# Separei as colunas em dois grupos: núméricas e categóricas.

# Identificando quais colunas são numéricas e quais são categóricas:
numeric_features = X.select_dtypes(include=['int64, float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

print(f"Colunas Numéricas: {list(numeric_features)}")
print(f"\nColuncas Categóricas: {list(categorical_features)}")

Colunas Numéricas: []

Coluncas Categóricas: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


In [22]:
# ---- 8. Pré-processamento (Criação do Pipeline de Transformação) ----

# Criei uma regra de preparação:
# -> StandardScaler: normaliza os números (coloca na mesma escala);
# -> OneHotEncoder: transforma as categorias (texto) em colunas de 0's e 1's.

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# O ColumnTransformer aplica o transformador certo na coluna certa
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

print("Pré-processador criado com sucesso!")

Pré-processador criado com sucesso!


In [23]:
# ---- 9. Divisão dos Dados em Treino e Teste ----
# Vou guardar 20% dos dados (teste) para o "exame final" do modelo e usarei 80% para o treino.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Dados divididos: {len(y_train)} para treino, {len(y_test)} para teste.")


Dados divididos: 32950 para treino, 8238 para teste.


In [24]:
# ---- 10. Criação e Treinamento do Modelo ----
# Criei o modelo, vou usar uma Pipeline para juntar o pré-processamento (Célula 8) e o modelo de Regressão Logística.
# O Comando .fit() vai iniciar o treino.
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

#Ensinando o modelo usando os dados de treino:
print("Iniciando o treinamento do modelo...")
model.fit(X_train, y_train)
print("Modelo treinado com suceso!")


Iniciando o treinamento do modelo...
Modelo treinado com suceso!


In [26]:
# ---- 11. Avaliação do Modelo (Acurácia e Relatório) ----
# Solictando ao modelo para fazer prevuisões nos dados de teste:
y_pred = model.predict(X_test)

# Acurácia
accuracy = accuracy_score(y_test, y_pred)
print("---📊 Resultados da Avaliação ---")
print(f"\nAcurácia (percentagem de acertos): {accuracy * 100:.2f}%")

# Relatório de Classificação
print("\nRelatório de Classificação Detalhado:")
print(classification_report(y_test, y_pred, target_names=['Não Converteu (0)', 'Converteu(1)']))

---📊 Resultados da Avaliação ---

Acurácia (percentagem de acertos): 89.63%

Relatório de Classificação Detalhado:
                   precision    recall  f1-score   support

Não Converteu (0)       0.90      0.99      0.94      7310
     Converteu(1)       0.65      0.17      0.27       928

         accuracy                           0.90      8238
        macro avg       0.78      0.58      0.61      8238
     weighted avg       0.88      0.90      0.87      8238



In [27]:
# ---- 12. Avaliação do Modelo (Matriz de Confusão) ----
print("\nMatriz de Confusão (Resumo de erros e acertos):")

#  [[Verdadeiros Negativos (Acertou 'Não'), Falsos Positivos (Errou, disse 'Sim')],
#  [Falsos Negativos (Errou, disse 'Não'), Verdadeiros Positivos (Acertou 'Sim')]]

print(confusion_matrix(y_test, y_pred))



Matriz de Confusão (Resumo de erros e acertos):
[[7223   87]
 [ 767  161]]


In [28]:
# ---- 13. Previsão de Novos Dados----
#Simulei um cliente novo para ver como o modelo se comporta na prática.

# Vamos simular um cliente novo (pode alterar estes valores):
cliente_novo = pd.DataFrame({
    'age': [45],
    'job': ['technician'],
    'marital': ['married'],
    'education': ['university.degree'],
    'default': ['no'],
    'housing': ['yes'],
    'loan': ['no'],
    'contact': ['cellular'],
    'month': ['may'],
    'day_of_week': ['fri'],
    'duration': [350],         # Duração da chamada
    'campaign': [1],           # Nº de contactos nesta campanha
    'pdays': [999],            # Dias desde o último contacto (999 = nunca)
    'previous': [0],           # Nº de contactos anteriores
    'poutcome': ['nonexistent'],
    'emp.var.rate': [-1.8],
    'cons.price.idx': [92.893],
    'cons.conf.idx': [-46.2],
    'euribor3m': [1.291],
    'nr.employed': [5099.1]
})

# Pedir ao modelo para prever
previsao_cliente = model.predict(cliente_novo)
probabilidade_cliente = model.predict_proba(cliente_novo) # Retorna [Prob de 0, Prob de 1]

print("--- 🔮 Teste de Previsão em Cliente Novo ---")
resultado = "CONVERTEU (1)" if previsao_cliente[0] == 1 else "NÃO CONVERTEU (0)"
print(f"Resultado da Previsão: {resultado}")

# Mostra a probabilidade de conversão (que é o segundo valor, índice [1])
print(f"Probabilidade de Conversão (Score): {probabilidade_cliente[0][1] * 100:.2f}%")


--- 🔮 Teste de Previsão em Cliente Novo ---
Resultado da Previsão: NÃO CONVERTEU (0)
Probabilidade de Conversão (Score): 9.45%
